In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import ast


In [2]:
# Load and process the CSV file
file_path = './data_new.csv'
df = pd.read_csv(file_path)

# Extract the input and output data
X = df[['bx', 'by', 'bz']].values
result_column = df['result']

# Convert the 'result' column from string to a list
y = np.array([ast.literal_eval(result_str) for result_str in result_column])

# Standardize the data
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)
print(X_train)


tensor([[ 0.1709, -0.6681,  1.4849],
        [-0.0363,  0.1496,  1.4720],
        [ 0.0461,  0.6052,  1.3903],
        ...,
        [ 0.2038, -0.6591,  1.5537],
        [ 0.3449,  0.7521, -0.4919],
        [ 0.6548,  0.9390,  0.6022]])


In [7]:
class BallToJointsNN(nn.Module):
    def __init__(self, input_size=3, output_size=11):
        super(BallToJointsNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 256)
        self.fc4 = nn.Linear(256, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x


model = BallToJointsNN()


In [9]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 500
batch_size = 32

# Create DataLoader for batch processing
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
val_dataset = torch.utils.data.TensorDataset(X_val, y_val)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_x, batch_y in train_loader:
        # Forward pass
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            val_loss += loss.item()
    val_loss /= len(val_loader)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')


Epoch [1/500], Train Loss: 0.0508, Val Loss: 0.1423
Epoch [2/500], Train Loss: 0.0484, Val Loss: 0.1360
Epoch [3/500], Train Loss: 0.0504, Val Loss: 0.1395
Epoch [4/500], Train Loss: 0.0505, Val Loss: 0.1359
Epoch [5/500], Train Loss: 0.0496, Val Loss: 0.1388
Epoch [6/500], Train Loss: 0.0515, Val Loss: 0.1340
Epoch [7/500], Train Loss: 0.0494, Val Loss: 0.1401
Epoch [8/500], Train Loss: 0.0503, Val Loss: 0.1349
Epoch [9/500], Train Loss: 0.0493, Val Loss: 0.1391
Epoch [10/500], Train Loss: 0.0504, Val Loss: 0.1416
Epoch [11/500], Train Loss: 0.0494, Val Loss: 0.1377
Epoch [12/500], Train Loss: 0.0482, Val Loss: 0.1337
Epoch [13/500], Train Loss: 0.0486, Val Loss: 0.1385
Epoch [14/500], Train Loss: 0.0486, Val Loss: 0.1346
Epoch [15/500], Train Loss: 0.0489, Val Loss: 0.1371
Epoch [16/500], Train Loss: 0.0473, Val Loss: 0.1357
Epoch [17/500], Train Loss: 0.0485, Val Loss: 0.1376
Epoch [18/500], Train Loss: 0.0483, Val Loss: 0.1360
Epoch [19/500], Train Loss: 0.0489, Val Loss: 0.1372
Ep

In [10]:
# Evaluation on validation set
model.eval()
with torch.no_grad():
    val_outputs = model(X_val)
    val_loss = criterion(val_outputs, y_val).item()
    val_mae = torch.mean(torch.abs(val_outputs - y_val)).item()

print(f'Validation Loss: {val_loss}, Validation MAE: {val_mae}')


Validation Loss: 0.145483136177063, Validation MAE: 0.13666115701198578


In [11]:
# Example new ball position
new_ball_position = np.array([[0.5, 0.5, 0.5]])
new_ball_position_scaled = scaler_X.transform(new_ball_position)
new_ball_position_tensor = torch.tensor(new_ball_position_scaled, dtype=torch.float32)

model.eval()
with torch.no_grad():
    predicted_joint_positions_scaled = model(new_ball_position_tensor).numpy()
    predicted_joint_positions = scaler_y.inverse_transform(predicted_joint_positions_scaled)

# Reshape the output into a 2D array
num_joints = 11
reshaped_output = predicted_joint_positions.reshape(num_joints)

print('Predicted Joint Positions:', reshaped_output)


Predicted Joint Positions: [-3.0653977e-01  7.2027266e-01  3.1409409e+00  5.4558307e-01
  1.0412589e-02  6.2268054e-01 -9.5393090e-04  5.2785683e-01
  8.2540553e-04  8.0568182e-01  1.2815006e+00]


In [12]:
import pickle

# Save the model to a pickle file
model_path = './model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(model, f)
